### Imports

In [ ]:
# Source for some of unet code which was adapted into this notebook: https://github.com/zhixuhao/unet

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

import skimage.io as io
import skimage.transform as trans
from skimage.transform import resize
from skimage import morphology, measure, draw, io, exposure

from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
from keras.layers import *
from keras.optimizers import *

### Specify parameters and directories

In [ ]:
UNET_WEIGHTS_VERSION = "01" # weight version as per hdf5 filename of desired unet

# Directories
INPUT_DIR = "" # directory containing entire dataset of radiographs (not just those for unet training/testing)
SAVE_DIR = "" # directory to save output segmented images in and other outputs
## load dataframes from an Excel file listing all radiographs in column 1 "filenames" (column 2 = "labels")
## update file path accordingly
ALL_RADIOGRAPHS_DF = pd.read_excel("all_radiographs.xlsx", dtype=str) 
os.chdir(INPUT_DIR)

# Parameters
START_INDEX = 0 # start index of radiographs you wish to process
STOP_INDEX = len(ALL_RADIOGRAPHS_DF) # stop index of radiographs you wish to process
THRESHOLD = 0.5 # threshold applied to unet mask predictions to generate boolean array

### Define for generating unet masks

In [ ]:
Implant = [128,128,128]
Unlabelled = [128,0,0]
COLOR_DICT = np.array([Implant, Unlabelled])

def predGenerator(dataframe, directory, x_col='filenames', target_size = (512,512), 
                  flag_multi_class = False,as_gray = True):
    os.chdir(directory)
    test_image_filenames = ALL_RADIOGRAPHS_DF[x_col]
    for test_image in test_image_filenames[START_INDEX:STOP_INDEX]:
        img = io.imread(test_image, as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,))
        img = np.reshape(img,(1,)+img.shape)
        yield img

def processPredictions(dataframe,directory,npyfile,x_col = 'filenames',num_class = 2):
    os.chdir(directory)
    test_image_filenames = dataframe[x_col]
    for filename, npimage in zip(tqdm(test_image_filenames[START_INDEX:STOP_INDEX]), 
                                 npyfile):
        # read in radiograph as numpy array
        original_xray = io.imread(f"{INPUT_DIR}/{filename}")
        target_shape = original_xray.shape
        
        # threshold unet generated mask
        mask = npimage[:,:,0]
        mask_thresh = (mask > THRESHOLD).astype(np.float64)
        
        # Remove small objects from mask and fill in holes
        mask_labelled = np.array(measure.label(mask_thresh))
        props = measure.regionprops(mask_labelled)
        try:
            minimum_object_size = 600
            mask_labelled = morphology.remove_small_objects(mask_labelled.astype(np.uint16), 
                                                           min_size=minimum_object_size)
            mask_thresh = morphology.remove_small_holes(mask_labelled.astype(np.uint16), 
                                                       area_threshold=300)
        except:
            print(f"Error remove small object / small holes for file: {filename}")
            pass
        
        # mask sure mask is the same size as the radiograph
        mask_thresh_resized = resize(mask_thresh, target_shape, anti_aliasing=False)
        
        # Use mask to crop implant out of radiograph, then apply adaptive histogram equilasation, and save output
        props_new = measure.regionprops(mask_thresh_resized.astype(np.uint16))
        try:
            bound_box_corners = props_new[0].bbox
            min_row, min_col, max_row, max_col = bound_box_corners
            try:
                img_cropped = ((original_xray*mask_thresh_resized).astype(np.uint16))[min_row-10:max_row+10,
                                                                                     min_col-10:max_col+10]
            except:
                img_cropped = ((original_xray*mask_thresh_resized).astype(np.uint16))[min_row:max_row, 
                                                                                     min_col:max_col]
            img_equalised_cropped = exposure.equalize_adapthist(img_cropped, clip_limit=0.03)
            io.imsave(os.path.join(directory,f"{filename[:-9]}CROP.png"), img_equalised_cropped)       
        except:
            print(f"Error producing bounding box for file: {filename}")

### Build unet

In [ ]:
def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
     
    return model

### Generate unet predicted masks

In [ ]:
predGene = predGenerator(ALL_RADIOGRAPHS_DF, INPUT_DIR)
model = unet()
model.load_weights(f"unet_implant_{UNET_WEIGHTS_VERSION}_FINALEPOCH.hdf5")
predictions = model.predict_generator(predGene,STOP_INDEX-START_INDEX,verbose=1)

### Using masks to segment radiographs, applying further final processing, and saving segmented images

In [ ]:
processPredictions(ALL_RADIOGRAPHS_DF, SAVE_DIR, predictions)